In [1]:
import pandas as pd

In [2]:
fz_rfg_processed_poultry = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Fz_Rfg Processed Poultry_POS_2020.xlsx')
fz_rfg_processed_poultry2 = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Fz_Rfg Processed Poultry_POS_2021.xlsx')
fz_rfg_processed_poultry3 = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Fz_Rfg Processed Poultry_POS_2022.xlsx')
fz_rfg_processed_poultry4 = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Fz_Rfg Processed Poultry_POS_2023.xlsx')
fz_rfg_processed_poultry5 = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Fz_Rfg Processed Poultry_POS_2024.xlsx')
product_attribute = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Cleaned Data/Product Attributes.xlsx')


In [3]:
# Imputation
def imputation_process(row):
    row['Incremental Units'] = row['Unit Sales'] - row['Base Unit Sales']
    row['Incremental Volume'] = row['Volume Sales'] - row['Base Volume Sales']
    row['Incremental Dollars'] = row['Dollar Sales'] - row['Base Dollar Sales']
    return row

# Split Geography column and only save its location
def extract_location(col):
    return col.split(' - ')[0]

# convert time into year month and day
def parse_time(col):
    # Year, month, day
    sep_data = col.split('-')
    year = int('20' + sep_data[2])
    month = int(sep_data[0][-2:])
    day = int(sep_data[1])
    # Quarter of given date
    quarter = (month - 1) // 3 + 1
    return pd.Series([year, quarter, month], index = ['Year', 'Quarter', 'Month'])

# Split product column and only save company name and product name
def parse_product(col):  
    return col.split(' - ')[0]


# Based on length of company name, select product name
def extract_product_name(full_string, brand_name):
    brand_name_length = len(brand_name)
    words = full_string[brand_name_length:].strip()
    return words

# # Arrange columns order
def change_column_order(df):
    final_order = [
        'Geography', 'Year', 'Quarter', 'Month', 'Season', 'Manufacturer Name',
        'Brand Name', 'Product Name', 'Flavor / Scent', 'Meat Source',
        'Unit Sales', 'Volume Sales', 'Dollar Sales', 'Price per Unit', 
        'Price per Volume', 'Base Unit Sales', 'Base Volume Sales', 'Base Dollar Sales',
        'Incremental Units', 'Incremental Volume', 'Incremental Dollars', 
        'ACV Weighted Distribution'
    ]
    return df[final_order]

def merge_content(df1, df2):
    combined = df1.merge(df2, how = 'left', on = 'Product')
    return combined

def map_season(month):
    # Northern Hemisphere standard mapping
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'


# Full tranform process
def transform_data(df, df2):
    table1 = merge_content(df, df2)
    # Used customized function
    table1['Geography'] = table1['Geography'].map(extract_location)
    table1[['Year', 'Quarter', 'Month']] = table1['Time'].apply(parse_time)
    table1['Product'] = table1['Product'].map(parse_product)
    table1['Product Name'] = table1.apply(lambda row: extract_product_name(row['Product'], row['Brand Name']), axis=1)
    #df['Product Name'] = df.apply(extract_product_name(df['Product'], df['Brand']), axis = 1)
    
    # Mapping quarter to seasonality
    table1['Season'] = table1['Month'].apply(map_season)
    # Change column order
    transformed = change_column_order(table1).apply(imputation_process, axis = 1)
    return transformed

In [4]:
fz_rfg_processed_poultry_2020 = transform_data(fz_rfg_processed_poultry, product_attribute)
fz_rfg_processed_poultry_2021 = transform_data(fz_rfg_processed_poultry2, product_attribute)
fz_rfg_processed_poultry_2022 = transform_data(fz_rfg_processed_poultry3, product_attribute)
fz_rfg_processed_poultry_2023 = transform_data(fz_rfg_processed_poultry4, product_attribute)
fz_rfg_processed_poultry_2024 = transform_data(fz_rfg_processed_poultry5, product_attribute)


In [9]:
fz_rfg_processed_poultry_2020

,Geography,Year,Quarter,Brand,Product Name,Unit Sales,Volume Sales,Dollar Sales,Price per Unit,Price per Volume,Base Unit Sales,Base Volume Sales,Base Dollar Sales,Incremental Units,Incremental Volume,Incremental Dollars,ACV Weighted Distribution
0,Total US,2020,1,MEAL MART,FROZEN REGULAR CHICKEN AND TURKEY CUTLET 21 OZ,52.792953,69.290750,651.207146,12.335115,9.398183,52.792953,69.290750,651.207146,0.000000,0.000000,0.000000,0.095009
1,Total US,2020,1,MEAL MART,FROZEN REGULAR CHICKEN AND TURKEY CUTLET 21 OZ,10.114684,13.275523,96.391946,9.529902,7.260878,6.865720,9.011257,83.096658,3.248964,4.264265,13.295288,0.019309
2,Total US,2020,1,MEAL MART,FROZEN REGULAR CHICKEN AND TURKEY NUGGET 24 OZ,50.728029,76.092043,689.735093,13.596726,9.064484,49.817002,74.725502,695.460184,0.911027,1.366541,-5.725091,0.161117
3,Total US,2020,1,DARTAGNAN,REFRIGERATED REGULAR DUCK WHOLE 5 OZ,22.842603,7.138313,913.475683,39.990000,127.968000,22.842603,7.138313,913.475683,0.000000,0.000000,0.000000,0.147815
4,Total US,2020,1,MAPLE LEAF FARMS,FROZEN REGULAR DUCK 1/2 14 OZ,1809.360250,1583.190219,21658.331405,11.970160,13.680183,1790.456257,1566.649224,21458.189147,18.903993,16.540994,200.142258,2.064006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362795,West,2021,1,PRIVATE LABEL,REFRIGERATED REGULAR TURKEY MEATBALL 12 OZ,1025.609440,769.207080,4566.857176,4.452823,5.937097,1005.045503,753.784127,4493.881936,20.563936,15.422952,72.975240,15.225621
362796,West,2021,1,FARM RICH,FROZEN REGULAR TURKEY MEATBALL 28 OZ,974.143150,1704.750512,5375.289861,5.517967,3.153124,724.950187,1268.662829,4160.397303,249.192963,436.087683,1214.892559,5.756330
362797,West,2021,1,SIGNATURE BY ROBERT,IRVINE REFRIGERATED REGULAR TURKEY BREAST 52 OZ,1.000000,3.250000,23.570000,23.570000,7.252308,1.000000,3.250000,23.570000,0.000000,0.000000,0.000000,0.009508
362798,West,2021,1,ROSINA,FROZEN REGULAR TURKEY MEATBALL 26 OZ,394.712878,641.408426,2067.185474,5.237188,3.222885,266.638531,433.287612,1722.332936,128.074347,208.120814,344.852538,8.246063


In [12]:
# Combined & export
full_fz_rfg_processed_poultry = pd.concat([fz_rfg_processed_poultry_2020, fz_rfg_processed_poultry_2021, 
                                           fz_rfg_processed_poultry_2022, fz_rfg_processed_poultry_2023, 
                                           fz_rfg_processed_poultry_2024])
output_file = '/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/ffull_rfg_processed_poultry.csv'
full_fz_rfg_processed_poultry.to_csv(output_file, index=False)